In [ ]:
import sqlite3
import pandas as pd

# Conectar a la base de datos
conexion = sqlite3.connect('../../01_data/Normativa_SQL.db')

In [ ]:
df = pd.read_csv('../../01_data/DB_CSV/tblReferencias.csv', decimal=';', encoding='utf-8')

# Añadir el DataFrame como una nueva tabla
df.to_sql('nombre_de_nueva_tabla', conexion, if_exists='replace', index=False)

7

In [2]:
# Crear un cursor para ejecutar consultas
cursor = conexion.cursor()

In [6]:
# Renombrar la tabla
cursor.execute("ALTER TABLE nombre_de_nueva_tabla RENAME TO tblReferencias;")

# Confirmar los cambios
conexion.commit()

In [7]:

# Ver las tablas disponibles
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas = cursor.fetchall()
print("Tablas en la base de datos:", tablas)

Tablas en la base de datos: [('tblDocAdj',), ('tblNormativa',), ('tblNormativaReferencias',), ('tblReferencias',)]


In [8]:
# Ver los campos de una tabla específica
nombre_tabla = 'tblNormativa'
cursor.execute(f"PRAGMA table_info({nombre_tabla});")
columnas = cursor.fetchall()
print(f"Columnas de la tabla {nombre_tabla}:", columnas)

Columnas de la tabla tblNormativa: [(0, 'IdNormativa', 'INTEGER', 0, None, 0), (1, 'TipoNormativa', 'TEXT', 0, None, 0), (2, 'Titulo', 'TEXT', 0, None, 0), (3, 'OrganoResp', 'INTEGER', 0, None, 0), (4, 'IdAprobadoPorOrg', 'REAL', 0, None, 0), (5, 'AprobadoPorOrg', 'TEXT', 0, None, 0), (6, 'IdEntidad', 'REAL', 0, None, 0), (7, 'Entidad', 'TEXT', 0, None, 0), (8, 'IdEstructuraResponsable', 'REAL', 0, None, 0), (9, 'EstructuraResponsable', 'REAL', 0, None, 0), (10, 'Caracter', 'TEXT', 0, None, 0), (11, 'FAprobado', 'TEXT', 0, None, 0), (12, 'FEntradaVigor', 'TEXT', 0, None, 0), (13, 'Descripcion', 'TEXT', 0, None, 0), (14, 'Documento', 'REAL', 0, None, 0), (15, 'BOUPV', 'TEXT', 0, None, 0), (16, 'Numero', 'REAL', 0, None, 0), (17, 'TextoConsolidado', 'INTEGER', 0, None, 0), (18, 'Vigente', 'INTEGER', 0, None, 0), (19, 'FFinVigencia', 'TEXT', 0, None, 0), (20, 'CreadoPor', 'TEXT', 0, None, 0), (21, 'FCreado', 'TEXT', 0, None, 0), (22, 'ModificadoPor', 'TEXT', 0, None, 0), (23, 'FModificado

In [9]:
# Crear la tabla tblActuales si no existe (ajusta las columnas según tu tabla)
cursor.execute("""
    CREATE TABLE IF NOT EXISTS tblActuales AS
    SELECT * FROM tblNormativa WHERE 0;
""")

# Insertar las filas donde FFinVigencia está vacío
cursor.execute("""
    INSERT INTO tblActuales
    SELECT * FROM tblNormativa
    WHERE FFinVigencia IS NULL OR FFinVigencia = '';
""")

# Confirmar los cambios
conexion.commit()

# Verificar si se copiaron correctamente
cursor.execute("SELECT * FROM tblActuales;")
filas = cursor.fetchall()
print(f"Se copiaron {len(filas)} filas a tblActuales.")


Se copiaron 1031 filas a tblActuales.


In [10]:
# Contar filas donde FFinVigencia tiene algún valor
cursor.execute("""
    SELECT COUNT(*) 
    FROM tblNormativa 
    WHERE FFinVigencia IS NOT NULL AND FFinVigencia != '';
""")

# Obtener el resultado
num_filas_con_valor = cursor.fetchone()[0]

print(f"Número de filas con FFinVigencia: {num_filas_con_valor}")

Número de filas con FFinVigencia: 606


In [27]:

# Definir los nombres de las tablas
tabla_normativas = "tblActuales"  # Tabla con los IDs de PDFs de interés
tabla_rutas = "tblDocAdj"  # Tabla con la información de carpetas y nombres de PDFs
tabla_destino = "tblRutaActuales"  # Nueva tabla donde guardaremos los resultados

In [34]:
# Crear la tabla de destino si no existe
cursor.execute(f"""
    CREATE TABLE IF NOT EXISTS {tabla_destino} (
        IdNormativa INTEGER,
        IdDocAdj INTEGER,
        Ruta_Doc TEXT NOT NULL,
        PRIMARY KEY (IdNormativa, IdDocAdj)
    )
""")
conexion.commit()

In [35]:
# Obtener los PDFs de interés (IDs) desde la tabla de normativas
cursor.execute(f"SELECT IdNormativa FROM {tabla_normativas}")
ids_interes = [row[0] for row in cursor.fetchall()]

In [36]:

# Obtener las rutas correspondientes
cursor.execute(f"""
    SELECT IdNormativa,IdDocAdj, Ruta, Documento 
    FROM {tabla_rutas} 
    WHERE IdNormativa IN ({','.join('?' * len(ids_interes))})
""", ids_interes)

In [39]:
import os
# Generar las rutas en el formato deseado utilizando os.path.join
datos_insertar = []
for id_normativa, id_documento, carpeta, nombre_documento in cursor.fetchall():
    # Construir la ruta utilizando os.path.join
    ruta = os.path.join(carpeta, nombre_documento)
    # Reemplazar las barras invertidas por barras diagonales
    ruta = ruta.replace("\\", "/")
    datos_insertar.append((id_normativa, id_documento, ruta))

# Insertar los datos en la nueva tabla
cursor.executemany(f"INSERT INTO {tabla_destino} (IdNormativa, IdDocAdj, Ruta_Doc) VALUES (?, ?,?)", datos_insertar)
conexion.commit()

print(f"Se han insertado {len(datos_insertar)} registros en '{tabla_destino}'.")

Se han insertado 1362 registros en 'tblRutaActuales'.


In [40]:
# Consulta todos los datos de la tabla
cursor.execute("SELECT * FROM tblRutaActuales")
resultados = cursor.fetchall()

# Mostrar resultados
for fila in resultados:
    print(fila)

(1, 1, '/1968/1968000000000100000001.pdf')
(6, 9, '/2011/2011000000000600000009.pdf')
(6, 10, '/2011/2011000000000600000010.pdf')
(6, 11, '/2011/2011000000000600000011.pdf')
(8, 14, '/1971/1971000000000800000014.pdf')
(8, 15, '/1971/1971000000000800000015.pdf')
(9, 16, '/1985/1985000000000900000016.pdf')
(9, 17, '/1985/1985000000000900000017.pdf')
(9, 18, '/1985/1985000000000900000018.pdf')
(9, 19, '/1985/1985000000000900000019.pdf')
(9, 20, '/1985/1985000000000900000020.rtf')
(9, 21, '/1985/1985000000000900000021.pdf')
(10, 22, '/2006/2006000000001000000022.pdf')
(11, 23, '/2006/2006000000001100000023.pdf')
(12, 24, '/2006/2006000000001200000024.pdf')
(13, 25, '/2006/2006000000001300000025.pdf')
(19, 31, '/2006/2006000000001900000031.pdf')
(20, 32, '/2006/2006000000002000000032.pdf')
(21, 33, '/2006/2006000000002100000033.pdf')
(24, 36, '/2006/2006000000002400000036.pdf')
(25, 37, '/2006/2006000100002500000037.pdf')
(26, 38, '/2006/2006000100002600000038.pdf')
(30, 42, '/2006/20060001

In [45]:
# Contar filas donde FFinVigencia tiene algún valor
cursor.execute("""
SELECT COUNT(*) 
FROM tblDocAdj 
WHERE Documento LIKE '%.rtf';
""")

# Obtener el resultado
num_filas_con_valor = cursor.fetchone()[0]

print(f"Número de filas con FFinVigencia: {num_filas_con_valor}")

Número de filas con FFinVigencia: 3


In [5]:
# Cerrar la conexión
conexion.close()

In [33]:
cursor.execute(f"DROP TABLE IF EXISTS {tabla_destino}")  # Elimina la tabla existente
conexion.commit()